## 2.1 Mini-batch gradient descent
- **Epoch**: a single pass through the whole training set.

- Variants
    - **(Batch) gradient descent**: whole training set at each training step.
    - **Mini-batch gradient descent**: `batch_size` samples at each training step. `batch_size` is a **hyper-parameter** (which should be finetuned) and commonly used value is 32, 64, 128, 256, 512.
    - **Stochastic gradient descent**: one sample at each training step.

- Rule of thumb
    - When the size of training set is less than 2000, use batch gradient descent.
    - Otherwise, use mini-batch gradient descent.

## 2.2 Variants of gradient descent

References:
http://ruder.io/optimizing-gradient-descent/index.html